In [1]:
import torch
assert torch.__version__>='1.2.0', 'Expect PyTorch>=1.2.0 but get {}'.format(torch.__version__)
from torch import nn
import torch.nn.functional as F

import numpy as np

import time
import os
import sys
import pickle

imp_dir = '../Implementations'
sys.path.insert(1, imp_dir)
data_dir = '../Data/criteo'
sys.path.insert(1, data_dir)

import logging
import importlib
importlib.reload(logging)

logging.basicConfig(stream=sys.stdout, format='%(asctime)s %(levelname)-6s %(message)s', level=logging.INFO, datefmt='%H:%M:%S')
logger = logging.getLogger(__name__)

In [2]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info('Device in Use: {}'.format(DEVICE))

18:53:06 INFO   Device in Use: cuda


## Prepare Data

### List all available files

In [3]:
np_artifact_dir = os.path.join(data_dir, 'criteo_train_numpy_artifact')
index_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='index', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
value_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='value', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))
label_artifact = sorted(list(filter(lambda x: x.split('-')[1]=='label', os.listdir(np_artifact_dir))), key = lambda x: int(x.split('.')[0].split('-')[-1]))

In [4]:
start = time.time()

train_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[:10]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[:10]]),
)

logger.info('Training data loaded after {:.2f}s'.format(time.time()-start))

start = time.time()

test_data = (
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in index_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in value_artifact[10:]]),
    np.vstack([np.load(os.path.join(np_artifact_dir, f)) for f in label_artifact[10:]]),
)

logger.info('Test data loaded after {:.2f}s'.format(time.time()-start))

18:55:36 INFO   Training data loaded after 148.22s
18:55:46 INFO   Test data loaded after 9.45s


## Create Model

In [19]:
import execution
importlib.reload(execution)
import DeepFM_BinClf_Torch 
importlib.reload(DeepFM_BinClf_Torch)

<module 'DeepFM_BinClf_Torch' from '../Implementations/DeepFM_BinClf_Torch.py'>

In [20]:
embedding_map_dict_pkl_path = os.path.join(data_dir, 'criteo_feature_dict_artifact/categorical_feature_map_dict.pkl')
with open(embedding_map_dict_pkl_path, 'rb') as f:
    embedding_map_dict = pickle.load(f)

In [21]:
DeepFM = DeepFM_BinClf_Torch.DeepFM_2D_Layer(len(embedding_map_dict)+60,
                                             39,
                                             10, 
                                             [400 for _ in range(3)], 
                                             [0, 0], 
                                             [0.5 for _ in range(4)]).to(DEVICE)

## Sample Run

In [22]:
cwd = os.getcwd()
checkpoint_dir = os.path.join(cwd, 'DeepFM_artifact')
checkpoint_prefix = 'DeepFM'

In [23]:
execution.train_model(DeepFM, 
                      train_data, 
                      test_data, 
                      F.binary_cross_entropy_with_logits, 
                      torch.optim.Adam(DeepFM.parameters()), 
                      DEVICE, 
                      checkpoint_dir, 
                      checkpoint_prefix)

19:47:54 INFO   Epoch 1/5 - Batch 1000/16000 Done - Train Loss: 0.475276, Val Loss: 0.474818
19:48:21 INFO   Epoch 1/5 - Batch 2000/16000 Done - Train Loss: 0.465876, Val Loss: 0.462235
19:48:47 INFO   Epoch 1/5 - Batch 3000/16000 Done - Train Loss: 0.461062, Val Loss: 0.458874
19:49:13 INFO   Epoch 1/5 - Batch 4000/16000 Done - Train Loss: 0.459669, Val Loss: 0.465313
19:49:39 INFO   Epoch 1/5 - Batch 5000/16000 Done - Train Loss: 0.458742, Val Loss: 0.467710
19:50:05 INFO   Epoch 1/5 - Batch 6000/16000 Done - Train Loss: 0.457884, Val Loss: 0.461592
19:50:31 INFO   Epoch 1/5 - Batch 7000/16000 Done - Train Loss: 0.457315, Val Loss: 0.452011
19:50:57 INFO   Epoch 1/5 - Batch 8000/16000 Done - Train Loss: 0.456881, Val Loss: 0.460217
19:51:24 INFO   Epoch 1/5 - Batch 9000/16000 Done - Train Loss: 0.456075, Val Loss: 0.453935
19:51:50 INFO   Epoch 1/5 - Batch 10000/16000 Done - Train Loss: 0.455462, Val Loss: 0.455011
19:52:16 INFO   Epoch 1/5 - Batch 11000/16000 Done - Train Loss: 0.45

## Model Performance

In [24]:
os.listdir(checkpoint_dir)

['DeepFM_1', 'DeepFM_2', 'DeepFM_5', 'DeepFM_4', 'DeepFM_3']

In [25]:
model = DeepFM_BinClf_Torch.DeepFM_2D_Layer(len(embedding_map_dict)+60,
                                             39,
                                             10, 
                                             [400 for _ in range(3)], 
                                             [0, 0], 
                                             [0.5 for _ in range(4)]).to(DEVICE)
model.load_state_dict(torch.load(os.path.join(checkpoint_dir, 'DeepFM_2')))
model = model.to(DEVICE)

In [26]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score(model, train_data, DEVICE)))

20:29:43 INFO   Model ROC Score: 0.819064


In [27]:
logger.info('Model ROC Score: {:.6f}'.format(execution.get_roc_auc_score(model, test_data, DEVICE)))

20:30:42 INFO   Model ROC Score: 0.801416
